# Project Report
Group 60: Daniyal, Ruth, Thomas, Paul

## Title

### Introduction

### Methods & Results
Our analysis makes use of the `repr`, `tidyverse`, and `tidymodels` packages in R.

In [1]:
# Setup
library(repr)
library(tidyverse)
library(tidymodels)
options(repr.matrix.max.rows = 6)
options(repr.plot.width = 8, repr.plot.height = 7)
set.seed(9248)

── Attaching packages ─────────────────────────────────────── tidyverse 1.3.1 ──

✔ ggplot2 3.3.6     ✔ purrr   0.3.4
✔ tibble  3.1.7     ✔ dplyr   1.0.9
✔ tidyr   1.2.0     ✔ stringr 1.4.0
✔ readr   2.1.2     ✔ forcats 0.5.1

── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()

── Attaching packages ────────────────────────────────────── tidymodels 1.0.0 ──

✔ broom        1.0.0     ✔ rsample      1.0.0
✔ dials        1.0.0     ✔ tune         1.0.0
✔ infer        1.0.2     ✔ workflows    1.0.0
✔ modeldata    1.0.0     ✔ workflowsets 1.0.0
✔ parsnip      1.0.0     ✔ yardstick    1.0.0
✔ recipes      1.0.1     

── Conflicts ───────────────────────────────────────── tidymodels_conflicts() ──
✖ scales::discard() masks purrr::discard()
✖ dplyr::filter()   masks stats::filter()
✖ recipes::fixed()  masks stringr::fixed()
✖ dplyr::lag()      masks stats::lag()
✖ yardstick::spec() masks r

In [2]:
# Obtaining and cleaning data
heart_raw <- read_delim("https://archive.ics.uci.edu/ml/machine-learning-databases/heart-disease/new.data",
                        col_names = FALSE, na = "-9") |>
    separate(X7, c("X7", "X8"), sep = " ", convert = TRUE) #|>
    #mutate(X3 = na_if(X3, "-9")))
heart_raw
n_observations <- nrow(heart_raw) / 12
row_indexes <- tibble(id = rep(1:n_observations, each = 12), Y = rep(1:12, times = n_observations))
row_indexes
heart_indexed <- bind_cols(row_indexes, heart_raw)
heart_indexed
heart_wide <- heart_indexed |>
    pivot_wider(names_from = Y, values_from = X1:X8)
heart_wide
?pivot_wider

ERROR: Error in open.connection(structure(4L, class = c("curl", "connection"), conn_id = <pointer: 0x3d2>), : Timeout was reached: [archive.ics.uci.edu] SSL connection timeout


### Discussion

### References